<a href="https://colab.research.google.com/github/botingchen/ML_Project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter # to format currency on charts axis
import json
import csv
api_key = "c9d26a6782e25df032d62899e2bbe7ef"
# https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key=<<api_key>>&language=en-US
# https://api.themoviedb.org/3/discover/movie?api_key=###&primary_release_date.gte=2006-01-01&primary_release_date.lte=2006-12-31
dataset = []
for year in range(2016,2022):
  for page_num in range(1,51):
    response = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=c9d26a6782e25df032d62899e2bbe7ef&year='+ str(year) + '&page=' + str(page_num))
    dataset.append(response.json())
# response = requests.get('https://api.themoviedb.org/3/movie/335797/credits?api_key=c9d26a6782e25df032d62899e2bbe7ef&language=en-US')

In [ ]:
# highest_revenue = []
# highest_revenue.append(response_2016_2.json())
# # print(highest_revenue['page'])
#   # print(i)
# for column in highest_revenue:
#   for i in column['results']:
#     print(i)
# # print(highest_revenue)
id = []
original_title = []
genre_ids = []
original_language =[]
popularity = []
release_date = []
for i in range(0,300):
  for data in dataset[i]['results']:
    id.append(data['id'])
    original_title.append(data['original_title'])
    genre_ids.append(data['genre_ids'])
    original_language.append(data['original_language'])
    popularity.append(data['popularity'])
    release_date.append(data['release_date'])
# print(dataset[299])

In [ ]:
df = pd.DataFrame(list(zip(id, original_title, genre_ids, original_language, popularity, release_date)),columns =['TMDB_id', 'original_title', 'genre_ids', 'original_language', 'popularity', 'release_date'])
temp_1 = []
### get cast ###
for index in df['TMDB_id']:
  response = requests.get('https://api.themoviedb.org/3/movie/' + str(index) + '/credits?api_key=c9d26a6782e25df032d62899e2bbe7ef&language=en-US')
  temp_1.append(response.json())

In [ ]:
cast_name = [None] * 6000
cast_id = [None] * 6000
cast_popularity = [None] * 6000
for iter in range(0,6000):
  count = 0
  cn = []
  ci = []
  cp = []
  for cast in temp_1[iter]['cast']:
    count += 1
    if count == 6:
      continue
    cn.append(cast['original_name'])
    ci.append(cast['id'])
    cp.append(cast['popularity'])
  cast_name[iter] = cn
  cast_id[iter] = ci
  cast_popularity[iter] = cp    
df['cast_name'] = cast_name
df['cast_id'] = cast_id
df['cast_popularity'] = cast_popularity


,TMDB_id,original_title,genre_ids,original_language,popularity,release_date,cast_name,cast_id,cast_popularity
0,102382,The Amazing Spider-Man 2,"[28, 12, 14]",en,1069.261,2014-04-16,"[Andrew Garfield, Emma Stone, Jamie Foxx, Dane...","[37625, 54693, 134, 122889, 10132, 13242, 35, ...","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,..."
1,557,Spider-Man,"[14, 28]",en,881.486,2002-05-01,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...","[2219, 5293, 205, 17051, 19153, 18999, 20580, ...","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,..."
2,559,Spider-Man 3,"[14, 28, 12]",en,884.269,2007-05-01,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[2219, 205, 17051, 19159, 17052, 18998, 18999,...","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,..."
3,335797,Sing,"[16, 35, 10751, 10402]",en,625.719,2016-11-23,"[Matthew McConaughey, Reese Witherspoon, Seth ...","[10297, 368, 52139, 1245, 4764, 1540615, 12094...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5..."
4,673,Harry Potter and the Prisoner of Azkaban,"[12, 14]",en,300.910,2004-05-31,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[10980, 10989, 10990, 1923, 5658, 64, 4566, 10...","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54..."


In [ ]:
### get revenue budget ...###
temp_2 = []
for index in df['TMDB_id']:
  response = requests.get('https://api.themoviedb.org/3/movie/' + str(index) + '?api_key=c9d26a6782e25df032d62899e2bbe7ef&language=en-US')
  temp_2.append(response.json())

In [ ]:
### put budget, runtime into dataframe ###
revenue = []
budget = []
runtime = []

for iter in range(len(temp_2)):
  revenue.append(temp_2[iter]['revenue'])
  budget.append(temp_2[iter]['budget'])
  runtime.append(temp_2[iter]['runtime'])

df['budget'] = budget
df['runtime'] = runtime

,TMDB_id,original_title,genre_ids,original_language,popularity,release_date,cast_name,cast_id,cast_popularity,budget,runtime
0,102382,The Amazing Spider-Man 2,"[28, 12, 14]",en,1069.261,2014-04-16,"[Andrew Garfield, Emma Stone, Jamie Foxx, Dane...","[37625, 54693, 134, 122889, 10132, 13242, 35, ...","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,...",200000000,142.0
1,557,Spider-Man,"[14, 28]",en,881.486,2002-05-01,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...","[2219, 5293, 205, 17051, 19153, 18999, 20580, ...","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,...",139000000,121.0
2,559,Spider-Man 3,"[14, 28, 12]",en,884.269,2007-05-01,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[2219, 205, 17051, 19159, 17052, 18998, 18999,...","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,...",258000000,139.0
3,335797,Sing,"[16, 35, 10751, 10402]",en,625.719,2016-11-23,"[Matthew McConaughey, Reese Witherspoon, Seth ...","[10297, 368, 52139, 1245, 4764, 1540615, 12094...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5...",75000000,108.0
4,673,Harry Potter and the Prisoner of Azkaban,"[12, 14]",en,300.910,2004-05-31,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[10980, 10989, 10990, 1923, 5658, 64, 4566, 10...","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54...",130000000,141.0


In [ ]:
company_name = []
company_id = []
for iter in range(len(temp_2)):
  count = 0
  con = []
  coi = []
  for company in temp_2[iter]['production_companies']:
    count += 1
    if count == 3:
      continue
    con.append(company['name'])
    coi.append(company['id'])
  company_name.append(con)
  company_id.append(coi)
df['company_name'] = company_name
df['company_id'] = company_id
df['revenue'] = revenue

genre_name = []
for iter in range(6000):
  gn = []
  for genre in temp_2[iter]['genres']:
    gn.append(genre['name'])
  genre_name.append(gn)
df['genre_name'] = genre_name

In [ ]:
rd = df['release_date']
season = []
for month in rd.str.split('-'):
  if month[1] == '01' or month[1] == '02' or month[1] == '12':
    season.append("winter")
  elif month[1] == '03' or month[1] == '04' or month[1] == '05':
    season.append("spring")
  elif month[1] == '06' or month[1] == '07' or month[1] == '08':
    season.append('summer')
  elif month[1] == '09' or month[1] == '10' or month[1] == '11':
    season.append('fall')
df['season'] = season
df.head()



,TMDB_id,original_title,genre_ids,original_language,popularity,release_date,cast_name,cast_id,cast_popularity,budget,runtime,company_name,company_id,revenue,Revenue_Level,original_language_label_encode,genre_name,season
0,102382,The Amazing Spider-Man 2,"[28, 12, 14]",en,1069.261,2014-04-16,"[Andrew Garfield, Emma Stone, Jamie Foxx, Dane...","[37625, 54693, 134, 122889, 10132, 13242, 35, ...","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,...",200000000,142.0,"[Marvel Entertainment, Columbia Pictures, Avi ...","[7505, 5, 166230]",708962323,A,6,"[Action, Adventure, Fantasy]",spring
1,557,Spider-Man,"[14, 28]",en,881.486,2002-05-01,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst, J...","[2219, 5293, 205, 17051, 19153, 18999, 20580, ...","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,...",139000000,121.0,"[Marvel Enterprises, Laura Ziskin Productions]","[19551, 326]",821708551,A,6,"[Fantasy, Action]",spring
2,559,Spider-Man 3,"[14, 28, 12]",en,884.269,2007-05-01,"[Tobey Maguire, Kirsten Dunst, James Franco, T...","[2219, 205, 17051, 19159, 17052, 18998, 18999,...","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,...",258000000,139.0,"[Laura Ziskin Productions, Marvel Studios]","[326, 420]",890871626,A,6,"[Fantasy, Action, Adventure]",spring
3,335797,Sing,"[16, 35, 10751, 10402]",en,625.719,2016-11-23,"[Matthew McConaughey, Reese Witherspoon, Seth ...","[10297, 368, 52139, 1245, 4764, 1540615, 12094...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5...",75000000,108.0,"[Illumination Entertainment, Universal Picture...","[6704, 33, 1778]",634151679,A,6,"[Animation, Comedy, Family, Music]",fall
4,673,Harry Potter and the Prisoner of Azkaban,"[12, 14]",en,300.910,2004-05-31,"[Daniel Radcliffe, Rupert Grint, Emma Watson, ...","[10980, 10989, 10990, 1923, 5658, 64, 4566, 10...","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54...",130000000,141.0,"[Warner Bros. Pictures, 1492 Pictures, P of A ...","[174, 436, 19092]",789804554,A,6,"[Adventure, Fantasy]",spring


In [3]:
import csv
import requests # to make TMDB API calls
import locale # to format currency as USD
locale.setlocale( locale.LC_ALL, '' )

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter # to format currency on charts axis
import json
df = pd.read_csv('/content/dataset_onehot.csv')

In [4]:
### do eight-level classification on revenue ###
from prettytable import PrettyTable
revenue_sl = [None] * 6000
count_F = 0
count_E = 0
count_D = 0
count_C = 0
count_B = 0
count_A = 0
count_S = 0
count_SS = 0
for i,rev in enumerate(df['revenue']):
  if df['revenue'][i] == 0:
    revenue_sl[i] = 'F'
    count_F += 1
  elif df['revenue'][i] > 0 and df['revenue'][i] <= 10000000:
    revenue_sl[i] = 'E'
    count_E += 1
  elif df['revenue'][i] > 10000000 and df['revenue'][i] <= 50000000:
    revenue_sl[i] = 'D'
    count_D += 1
  elif df['revenue'][i] > 50000000 and df['revenue'][i] <= 100000000:
    revenue_sl[i] = 'C'
    count_C += 1
  elif df['revenue'][i] > 100000000 and df['revenue'][i] <= 500000000:
    revenue_sl[i] = 'B'
    count_B += 1
  elif df['revenue'][i] > 500000000:
    revenue_sl[i] = 'A'
    count_A += 1

table = PrettyTable()
table.field_names = ["Revenue_Level","Count"]
table.add_row(["F",count_F])
table.add_row(["E",count_E])
table.add_row(["D",count_D])
table.add_row(["C",count_C])
table.add_row(["B",count_B])
table.add_row(["A",count_A])
table.add_row(["total",count_A + count_B + count_C + count_D + count_E + count_F])
print(table)
df['Revenue_Level'] = revenue_sl

+---------------+-------+
| Revenue_Level | Count |
+---------------+-------+
|       F       |  2412 |
|       E       |  842  |
|       D       |  839  |
|       C       |  450  |
|       B       |  1116 |
|       A       |  341  |
|     total     |  6000 |
+---------------+-------+


In [5]:
### fill the missing data ###
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['original_language_label_encode'] = labelencoder.fit_transform(df['original_language'])
df.isnull().sum()
df.fillna(int(df['runtime'].mean()), inplace=True)
df.isnull().sum()
df['season_label_encode'] = labelencoder.fit_transform(df['season'])
# p_write = df.to_csv('dataset.csv',mode = 'w',index = False)

In [6]:
### visualization ###
all_genre = []
for i in df['genre_name']:
  for ii in i:
    all_genre.append(ii)
all_genre = pd.DataFrame(all_genre)
all_genre.value_counts()
df['season'].value_counts()

fall      1894
summer    1605
winter    1322
spring    1179
Name: season, dtype: int64

In [8]:
### generate final dataset ### 
dataset = df.drop(['TMDB_id', 'original_title', 'genre_ids', 'original_language', 'release_date', 'cast_id', 'company_id', 'season'], axis=1)
### Control length of cast name, company name, genre name for limit total of feature###
dct = pd.Series(df['cast_name']).str.lower().str.replace("[","").str.replace("]","").str.replace("'","").str.split(",")
cast_name = []
for movie in dct:
  cn = []
  count = 0
  for actress_name in movie:
    count += 1
    if count <= 3:
      cn.append(actress_name)
    else:
      continue
  cast_name.append(cn)
dataset["cast_name"] = cast_name
dct = pd.Series(df['company_name']).str.lower().str.replace("[","").str.replace("]","").str.replace("'","").str.split(",")
company_name = []
for movie in dct:
  cn = []
  count = 0
  for temp_name in movie:
    count += 1
    if count <= 2:
      cn.append(temp_name)
    else:
      continue
  company_name.append(cn)
dataset["company_name"] = company_name
dct = pd.Series(df['genre_name']).str.lower().str.replace("[","").str.replace("]","").str.replace("'","").str.replace(" ","").str.split(",")
genre_name = []
for movie in dct:
  count = 0
  cc = []
  for temp_name in movie:
    count += 1
    if count <= 3:
      cc.append(temp_name)
    else:
      continue
  genre_name.append(cc)
dataset["genre_name"] = genre_name
dataset.head()

,popularity,cast_name,cast_popularity,budget,runtime,company_name,revenue,Revenue_Level,original_language_label_encode,genre_name,season_label_encode
0,1069.261,"[andrew garfield, emma stone, jamie foxx]","[59.496, 20.223, 11.873, 11.457, 2.659, 10.92,...",200000000,142.0,"[marvel entertainment, columbia pictures]",708962323,A,6,"[action, adventure, fantasy]",1
1,881.486,"[tobey maguire, willem dafoe, kirsten dunst]","[38.197, 27.696, 22.164, 12.303, 3.864, 19.57,...",139000000,121.0,"[marvel enterprises, laura ziskin productions]",821708551,A,6,"[fantasy, action]",1
2,884.269,"[tobey maguire, kirsten dunst, james franco]","[38.197, 22.164, 12.303, 11.794, 4.404, 8.422,...",258000000,139.0,"[laura ziskin productions, marvel studios]",890871626,A,6,"[fantasy, action, adventure]",1
3,625.719,"[matthew mcconaughey, reese witherspoon, set...","[15.356, 14.74, 6.437, 42.053, 6.211, 6.032, 5...",75000000,108.0,"[illumination entertainment, universal pictures]",634151679,A,6,"[animation, comedy, family]",0
4,300.910,"[daniel radcliffe, rupert grint, emma watson]","[30.499, 19.278, 46.894, 11.044, 13.221, 19.54...",130000000,141.0,"[warner bros. pictures, 1492 pictures]",789804554,A,6,"[adventure, fantasy]",1


In [9]:
### performing one hot ###
genre_onehot = pd.Series(dataset['genre_name']).str.join('|').str.get_dummies()
company_onehot = pd.Series(dataset['company_name']).str.join('|').str.get_dummies()
cast_onehot = pd.Series(dataset['cast_name']).str.join('|').str.get_dummies()
dataset_onehot = pd.concat([dataset,genre_onehot,company_onehot,cast_onehot],axis = 1)
dataset_onehot = dataset_onehot.drop(['cast_name', 'company_name', 'genre_name', 'cast_popularity', 'revenue'], axis=1)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
X = dataset_onehot.drop(['Revenue_Level'], axis=1)
y = pd.DataFrame(revenue_sl)
kf = KFold(n_splits=3, shuffle = True)
rf = RandomForestClassifier(n_estimators=100,bootstrap = True)
conf_matrix_list_of_arrays = []
score_list = []
recall_list = []
precision_list = []

for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].values
    y_train = y.iloc[train_index].values.ravel()
    X_test = X.iloc[test_index].values
    y_test = y.iloc[test_index].values.ravel()
    
    
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred_rf)
    recall = recall_score(y_test, y_pred_rf,average = None)
    precision = precision_score(y_test, y_pred_rf,average = None)
    conf_matrix = confusion_matrix(y_test, y_pred_rf)
    
    score_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)
    conf_matrix_list_of_arrays.append(conf_matrix)

print("="*30)
print("six-level classification on RandomForest(n_estimators=100)")
print("="*30)

print("Confusion matrix:\n",np.mean(conf_matrix_list_of_arrays,axis = 0))

table = PrettyTable()
table.field_names = ["Revenue_Level","Accuracy Score","Sensitivity(Recall) Score","Precision Score"]
table.add_row(["A",np.mean(score_list),(recall_list[0][0] + recall_list[1][0] + recall_list[2][0]) / 3,(precision_list[0][0] + precision_list[1][0] + precision_list[2][0]) / 3])
table.add_row(["B",np.mean(score_list),(recall_list[0][1] + recall_list[1][1] + recall_list[2][1]) / 3,(precision_list[0][1] + precision_list[1][1] + precision_list[2][1]) / 3])
table.add_row(["C",np.mean(score_list),(recall_list[0][2] + recall_list[1][2] + recall_list[2][2]) / 3,(precision_list[0][2] + precision_list[1][2] + precision_list[2][2]) / 3])
table.add_row(["D",np.mean(score_list),(recall_list[0][3] + recall_list[1][3] + recall_list[2][3]) / 3,(precision_list[0][3] + precision_list[1][3] + precision_list[2][3]) / 3])
table.add_row(["E",np.mean(score_list),(recall_list[0][4] + recall_list[1][4] + recall_list[2][4]) / 3,(precision_list[0][4] + precision_list[1][4] + precision_list[2][4]) / 3])
table.add_row(["F",np.mean(score_list),(recall_list[0][5] + recall_list[1][5] + recall_list[2][5]) / 3,(precision_list[0][5] + precision_list[1][5] + precision_list[2][5]) / 3])

print(table)

binary classification on RandomForest(n_estimators=100) w/o PCA
Confusion matrix:
 [[9.70000000e+01 1.63333333e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 3.33333333e-01]
 [5.66666667e+00 3.42666667e+02 2.66666667e+00 1.20000000e+01
  3.33333333e-01 8.66666667e+00]
 [0.00000000e+00 3.50000000e+01 8.06666667e+01 1.60000000e+01
  3.33333333e-01 1.80000000e+01]
 [0.00000000e+00 3.83333333e+01 1.66666667e+00 1.66000000e+02
  2.00000000e+00 7.16666667e+01]
 [0.00000000e+00 6.66666667e+00 3.33333333e-01 1.43333333e+01
  1.53333333e+02 1.06000000e+02]
 [3.33333333e-01 1.10000000e+01 6.66666667e-01 9.00000000e+00
  1.00000000e+00 7.82000000e+02]]
+-------+--------------------+---------------------------+--------------------+
| Class |   Accuracy Score   | Sensitivity(Recall) Score |  Precision Score   |
+-------+--------------------+---------------------------+--------------------+
|   A   | 0.8108333333333334 |     0.8533282031550596    | 0.9427031189834425 |
|   B   | 0.81083333333333

In [27]:
from sklearn.tree import DecisionTreeClassifier
##### Decision Tree #####
dstree = DecisionTreeClassifier(class_weight="balanced")

conf_matrix_list_of_arrays = []
score_list = []
recall_list = []
precision_list = []

##### Binary Classification w/o PCA #####
kf = KFold(n_splits=3, shuffle = True)
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].values
    y_train = y.iloc[train_index].values.ravel()
    X_test = X.iloc[test_index].values
    y_test = y.iloc[test_index].values.ravel()
    
    dstree.fit(X_train, y_train)
    y_pred_ds = dstree.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred_ds)
    recall = recall_score(y_test, y_pred_ds, average = None)
    precision = precision_score(y_test, y_pred_ds, average = None)
    conf_matrix = confusion_matrix(y_test, y_pred_ds)
    
    score_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)
    conf_matrix_list_of_arrays.append(conf_matrix)

print("="*30)
print("six-level classification on Decision Tree")
print("="*30)

print("Confusion matrix:\n",np.mean(conf_matrix_list_of_arrays,axis = 0))

table = PrettyTable()
table.field_names = ["Revenue_Level","Accuracy Score","Sensitivity(Recall) Score","Precision Score"]
table.add_row(["A",np.mean(score_list),(recall_list[0][0] + recall_list[1][0] + recall_list[2][0]) / 3,(precision_list[0][0] + precision_list[1][0] + precision_list[2][0]) / 3])
table.add_row(["B",np.mean(score_list),(recall_list[0][1] + recall_list[1][1] + recall_list[2][1]) / 3,(precision_list[0][1] + precision_list[1][1] + precision_list[2][1]) / 3])
table.add_row(["C",np.mean(score_list),(recall_list[0][2] + recall_list[1][2] + recall_list[2][2]) / 3,(precision_list[0][2] + precision_list[1][2] + precision_list[2][2]) / 3])
table.add_row(["D",np.mean(score_list),(recall_list[0][3] + recall_list[1][3] + recall_list[2][3]) / 3,(precision_list[0][3] + precision_list[1][3] + precision_list[2][3]) / 3])
table.add_row(["E",np.mean(score_list),(recall_list[0][4] + recall_list[1][4] + recall_list[2][4]) / 3,(precision_list[0][4] + precision_list[1][4] + precision_list[2][4]) / 3])
table.add_row(["F",np.mean(score_list),(recall_list[0][5] + recall_list[1][5] + recall_list[2][5]) / 3,(precision_list[0][5] + precision_list[1][5] + precision_list[2][5]) / 3])

print(table)

six-level classification on Decision Tree
Confusion matrix:
 [[9.76666667e+01 1.26666667e+01 1.66666667e+00 3.33333333e-01
  1.00000000e+00 3.33333333e-01]
 [1.90000000e+01 2.99000000e+02 2.03333333e+01 2.03333333e+01
  4.00000000e+00 9.33333333e+00]
 [1.00000000e+00 2.56666667e+01 8.70000000e+01 2.10000000e+01
  6.66666667e+00 8.66666667e+00]
 [6.66666667e-01 1.56666667e+01 2.26666667e+01 1.90000000e+02
  2.20000000e+01 2.86666667e+01]
 [3.33333333e-01 3.33333333e+00 5.66666667e+00 2.00000000e+01
  1.94000000e+02 5.73333333e+01]
 [1.00000000e+00 1.03333333e+01 8.00000000e+00 3.83333333e+01
  6.36666667e+01 6.82666667e+02]]
+---------------+--------------------+---------------------------+--------------------+
| Revenue_Level |   Accuracy Score   | Sensitivity(Recall) Score |  Precision Score   |
+---------------+--------------------+---------------------------+--------------------+
|       A       | 0.7751666666666667 |     0.8586785534153956    | 0.820060359694506  |
|       B       

In [32]:
import lightgbm as lgb
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
conf_matrix_list_of_arrays = []
score_list = []
recall_list = []
precision_list = []
num_fold = 0
params = {
    "metric": 'rmse',
    "verbosity": -1
}
##### Binary Classification w/o PCA #####
kf = KFold(n_splits=3, shuffle = True)
for train_index, test_index in kf.split(X):
    X_train = X.iloc[train_index].values
    y_train = y.iloc[train_index].values.ravel()
    X_test = X.iloc[test_index].values
    y_test = y.iloc[test_index].values.ravel()
    model_lgb = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
    model_lgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric='rmse',verbose=100, early_stopping_rounds=1000)

    y_pred_ds = model_lgb.predict(X_test)
    rmse_valid = np.sqrt(mean_squared_error(y_test, y_pred_ds))
    print("Fold" ,num_fold, "xvalid rmse:", rmse_valid)
    num_fold = num_fold + 1
    
    oof_rmse += rmse_valid

    predictions_test_lgb += model_lgb.predict(test_df[xtrain.columns])/num_of_splits

    # xgboostModel.fit(X_train, y_train)
    # y_pred_ds = xgboostModel.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred_ds)
    recall = recall_score(y_test, y_pred_ds, average = None)
    precision = precision_score(y_test, y_pred_ds, average = None)
    conf_matrix = confusion_matrix(y_test, y_pred_ds)
    
    score_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)
    conf_matrix_list_of_arrays.append(conf_matrix)

print("="*30)
print("six-level classification on XGBoost")
print("="*30)

print("Confusion matrix:\n",np.mean(conf_matrix_list_of_arrays,axis = 0))

table = PrettyTable()
table.field_names = ["Revenue_Level","Accuracy Score","Sensitivity(Recall) Score","Precision Score"]
table.add_row(["A",np.mean(score_list),(recall_list[0][0] + recall_list[1][0] + recall_list[2][0]) / 3,(precision_list[0][0] + precision_list[1][0] + precision_list[2][0]) / 3])
table.add_row(["B",np.mean(score_list),(recall_list[0][1] + recall_list[1][1] + recall_list[2][1]) / 3,(precision_list[0][1] + precision_list[1][1] + precision_list[2][1]) / 3])
table.add_row(["C",np.mean(score_list),(recall_list[0][2] + recall_list[1][2] + recall_list[2][2]) / 3,(precision_list[0][2] + precision_list[1][2] + precision_list[2][2]) / 3])
table.add_row(["D",np.mean(score_list),(recall_list[0][3] + recall_list[1][3] + recall_list[2][3]) / 3,(precision_list[0][3] + precision_list[1][3] + precision_list[2][3]) / 3])
table.add_row(["E",np.mean(score_list),(recall_list[0][4] + recall_list[1][4] + recall_list[2][4]) / 3,(precision_list[0][4] + precision_list[1][4] + precision_list[2][4]) / 3])
table.add_row(["F",np.mean(score_list),(recall_list[0][5] + recall_list[1][5] + recall_list[2][5]) / 3,(precision_list[0][5] + precision_list[1][5] + precision_list[2][5]) / 3])

print(table)

ValueError: ignored